In [1]:
import pandas as pd
import numpy as np

## Read csv from URL

In [2]:
csv_url = 'https://raw.githubusercontent.com/seidelj/python-course/main/week8/prices.csv'
df = pd.read_csv(csv_url)
df.head()

,period,price,session,market
0,1,214,online1,1
1,2,164,online1,1
2,3,161,online1,1
3,4,187,online1,1
4,5,289,online1,1


In [3]:
df.sort_values(by=['session', 'market', 'period'], inplace=True)
df.head()

,period,price,session,market
0,1,214,online1,1
1,2,164,online1,1
2,3,161,online1,1
3,4,187,online1,1
4,5,289,online1,1


## Summarizing Data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   period   200 non-null    int64 
 1   price    200 non-null    int64 
 2   session  200 non-null    object
 3   market   200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.8+ KB


In [5]:
df['price'].describe()

count     200.000000
mean      354.365000
std       224.972587
min         0.000000
25%       178.500000
50%       324.500000
75%       496.000000
max      1121.000000
Name: price, dtype: float64

In [6]:
df['session'].value_counts()

online1     20
online10    20
online2     20
online3     20
online4     20
online5     20
online6     20
online7     20
online8     20
online9     20
Name: session, dtype: int64

In [7]:
df['period'].value_counts()

1     20
2     20
3     20
4     20
5     20
6     20
7     20
8     20
9     20
10    20
Name: period, dtype: int64

In [8]:
df['market'].value_counts()

1    100
2    100
Name: market, dtype: int64

In [9]:
pd.pivot_table(df, columns='market', index= 'session', aggfunc="count")

period     price    
market        1   2     1   2
session                      
online1      10  10    10  10
online10     10  10    10  10
online2      10  10    10  10
online3      10  10    10  10
online4      10  10    10  10
online5      10  10    10  10
online6      10  10    10  10
online7      10  10    10  10
online8      10  10    10  10
online9      10  10    10  10

## Creating new variables using custom functions

In [10]:
def get_fv(row):
    # the fundemental value is the expected divident of the assset
    # times the number of remaining dividend payments
    ev=24
    result = (10-row['period']+1)*ev
    return result

In [11]:
df['fv'] = df.apply(get_fv, axis=1)
df

,period,price,session,market,fv
0,1,214,online1,1,240
1,2,164,online1,1,216
2,3,161,online1,1,192
3,4,187,online1,1,168
4,5,289,online1,1,144
...,...,...,...,...,...
195,6,806,online9,2,120
196,7,393,online9,2,96
197,8,388,online9,2,72
198,9,196,online9,2,48


In [12]:
def rad(df):
    abDiff = abs(df['price'] - df['fv'])
    fv_avg = df['fv'].mean()
    quotient = abDiff.divide(fv_avg)
    result = quotient.sum()/10
    return result

def npa(df):
    fv1 = df['fv'].tolist()[0]
    diff_series = (df['price'] - df['fv'])/fv1
    maxt = diff_series.max()
    mint = diff_series.min()
    result = maxt - min_t
    s = pd.Series(result, index=['df'])
    return result

In [13]:
groups = df.groupby(by=['session', 'market'])
groups.get_group(('online9', 2))

,period,price,session,market,fv
190,1,411,online9,2,240
191,2,525,online9,2,216
192,3,940,online9,2,192
193,4,1121,online9,2,168
194,5,604,online9,2,144
195,6,806,online9,2,120
196,7,393,online9,2,96
197,8,388,online9,2,72
198,9,196,online9,2,48
199,10,429,online9,2,24


In [14]:
agg_series = groups.apply(rad)
agg_df = agg_series.reset_index(name='rad')
agg_df

,session,market,rad
0,online1,1,0.729545
1,online1,2,1.424242
2,online10,1,2.543939
3,online10,2,2.334091
4,online2,1,1.883333
5,online2,2,1.398485
6,online3,1,3.259091
7,online3,2,2.210606
8,online4,1,1.628030
9,online4,2,2.946212


In [15]:
df = df.merge(agg_df, on=['session', 'market'])
df

,period,price,session,market,fv,rad
0,1,214,online1,1,240,0.729545
1,2,164,online1,1,216,0.729545
2,3,161,online1,1,192,0.729545
3,4,187,online1,1,168,0.729545
4,5,289,online1,1,144,0.729545
...,...,...,...,...,...,...
195,6,806,online9,2,120,3.403788
196,7,393,online9,2,96,3.403788
197,8,388,online9,2,72,3.403788
198,9,196,online9,2,48,3.403788


In [16]:
import os
csvFilePath = os.path.join(os.getcwd(), 'prices.csv')
df.to_csv(csvFilePath, index=False)